In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

def calculate_metrics(loader: DataLoader, model: nn.Module):
    y_true = []
    y_pred = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.numpy())
            y_true.extend(labels.numpy())

    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred, average='macro')
    precision = precision_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    return accuracy, recall, precision, f1

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data_dz1', train=True, download=True, transform=transform)
valset = torchvision.datasets.MNIST(root='./data_dz1', train=False, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
valloader = DataLoader(valset, batch_size=64, shuffle=False)


net = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01 )

val_acc, val_rec, val_prec, val_f1 =0,0,0,0
epoch = 0


while val_acc < 0.95 and val_rec < 0.95 and val_prec < 0.95 and val_f1 < 0.95:
    
    running_loss = 0.0
    net.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss:nn.CrossEntropyLoss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader):.3f}')

    train_acc, train_rec, train_prec, train_f1 = calculate_metrics(trainloader, net)
    print(f'Training - Accuracy: {train_acc}, Recall: {train_rec}, Precision: {train_prec}, F1 Score: {train_f1}')

    val_acc, val_rec, val_prec, val_f1 = calculate_metrics(valloader, net)
    print(f'Validation - Accuracy: {val_acc}, Recall: {val_rec}, Precision: {val_prec}, F1 Score: {val_f1}')

    epoch+=1

print('Finished Training')
# val acc 0.93 is limit

Epoch 1, Loss: 0.609
Training - Accuracy: 0.8856333333333334, Recall: 0.8843893463497624, Precision: 0.8862226794929979, F1 Score: 0.8846061019323572
Validation - Accuracy: 0.8934, Recall: 0.8921267340574273, Precision: 0.8931809791904781, F1 Score: 0.8922105985063539
Epoch 2, Loss: 0.387
Training - Accuracy: 0.8959166666666667, Recall: 0.8943218672701784, Precision: 0.8960305691702022, F1 Score: 0.8945584969493616
Validation - Accuracy: 0.9023, Recall: 0.9008880470591956, Precision: 0.9019454263436819, F1 Score: 0.9007884313481614
Epoch 3, Loss: 0.353
Training - Accuracy: 0.9046833333333333, Recall: 0.9036426542713627, Precision: 0.9039291833625424, F1 Score: 0.9035076273369421
Validation - Accuracy: 0.9089, Recall: 0.9079758438148671, Precision: 0.9079832417711279, F1 Score: 0.9077206235569971
Epoch 4, Loss: 0.335
Training - Accuracy: 0.9072333333333333, Recall: 0.9060764171220637, Precision: 0.906503176241252, F1 Score: 0.90600924856894
Validation - Accuracy: 0.9111, Recall: 0.91011